This tutorial will show you how to [install Milvus from source code](https://github.com/milvus-io/milvus/blob/master/INSTALL.md) and run the [python code](https://milvus-io.github.io/milvus-sdk-python/pythondoc/v0.2.14/index.html).

## Install Milvus from source code

### Install Camke

- Get CMake 3.12 or higher. Here we get CMake 3.12.

In [ ]:
! wget https://cmake.org/files/v3.12/cmake-3.12.2-Linux-x86_64.tar.gz

- Extract the file and install CMake.

In [ ]:
! tar zxvf cmake-3.12.2-Linux-x86_64.tar.gz
! mv cmake-3.12.2-Linux-x86_64 /opt/cmake-3.12.2
! ln -sf /opt/cmake-3.12.2/bin/* /usr/bin/

- Show the version.

In [ ]:
!cmake --version

### Download the code and Compile

Step 1 Download Milvus source code

In [ ]:
!git clone -b 0.10.2 https://github.com/milvus-io/milvus.git

Step 2 Install dependencies

In [ ]:
% cd /content/milvus/core
! ./ubuntu_build_deps.sh

Step 3 Build Milvus source code

In [ ]:
% cd /content/milvus/core  
!ls
!./build.sh -t Release
# If you want to build GPU version, add -g option, and Edit the Notebook settings with GPU
# !./build.sh -t Release -g

### Launch Milvus server

Add `lib/` directory to `LD_LIBRARY_PATH`

In [ ]:
% cd /content/milvus/core/milvus
! echo $LD_LIBRARY_PATH
import os
os.environ['LD_LIBRARY_PATH'] +=":/content/milvus/core/milvus/lib"
! echo $LD_LIBRARY_PATH

Start Milvus server and hang

In [ ]:
% cd scripts
! ls
! nohup ./start_server.sh &

Show Milvus server status

If you can find the following content, the Milvus server is successfully running: `Milvus server started successfully!`

In [ ]:
! ls
! cat nohup.out

## Run Python Code

### Install Milvus Python SDK

In [ ]:
! pip install pymilvus

### Import pymilvus

In [ ]:
import random
import numpy as np

from milvus import Milvus, IndexType, MetricType, Status

# Milvus server IP address and port.
# You may need to change _HOST and _PORT accordingly.
_HOST = '127.0.0.1'
_PORT = '19530'  # default value
# _PORT = '19121'  # default http value

# Vector parameters
_DIM = 8  # dimension of vector

_INDEX_FILE_SIZE = 32  # max file size of stored index

# the demo name.
collection_name = 'example_collection_'
partition_tag = 'demo_tag_'
segment_name= ''

# 10 vectors with 8 dimension, per element is float32 type, vectors should be a 2-D array
vectors = [[random.random() for _ in range(_DIM)] for _ in range(10)]
ids = [i for i in range(10)]

### Connect Milvus and check version

In [ ]:
# Connect to Milvus Server 
milvus = Milvus(_HOST, _PORT)

# Close client instance
# milvus.close()

# Returns the status of the Milvus server.
server_status = milvus.server_status(timeout=10)
print(server_status)

In [ ]:
# Returns the version of the client.
client_version= milvus.client_version()
print(client_version)

# Returns the version of the Milvus server.
server_version = milvus.server_version(timeout=10)
print(server_version)

### Check and create collection/index

In [ ]:
# Checks whether a collection exists.
milvus.has_collection(collection_name=collection_name, timeout=10)

In [ ]:
# Information needed to create a collection.Defult index_file_size=1024 and metric_type=MetricType.L2
param={'collection_name':collection_name, 'dimension': _DIM, 'index_file_size': _INDEX_FILE_SIZE, 'metric_type': MetricType.L2}
# Creates a collection.
milvus.create_collection(param, timeout=10)

In [ ]:
# Checks whether a partition exists.
milvus.has_partition(collection_name=collection_name, partition_tag=partition_tag)

In [ ]:
# Create a partition for a collection.
milvus.create_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=10)

In [ ]:
ivf_param = {'nlist': 16384}
# Creates index for a collection.
milvus.create_index(collection_name=collection_name, index_type=IndexType.IVF_FLAT, params=ivf_param)

### Insert data and flush

In [ ]:
# Insert vectors to a collection.
milvus.insert(collection_name=collection_name, records=vectors, ids=ids)

In [ ]:
# Flushes vector data in one collection or multiple collections to disk.
milvus.flush(collection_name_array=[collection_name], timeout=None)

### Load data and search

In [ ]:
# Loads a collection for caching.
milvus.load_collection(collection_name=collection_name, timeout=None)

In [ ]:
# Search vectors in a collection.
search_param = { "nprobe": 16 }
milvus.search(collection_name=collection_name,query_records=[vectors[0]],partition_tags=None,top_k=10,params=search_param)

### Get info about collection/index

In [ ]:
# Returns collection list.
milvus.list_collections(timeout=10)

In [ ]:
# Show all partitions in a collection.
milvus.list_partitions(collection_name=collection_name, timeout=10)

In [ ]:
# Returns the number of vectors in a collection.
milvus.count_entities(collection_name=collection_name, timeout=10)

In [ ]:
# Returns information of a collection.
milvus.get_collection_info(collection_name=collection_name, timeout=10)

In [ ]:
# Returns collection statistics information
milvus.get_collection_stats(collection_name=collection_name, timeout=10)

In [ ]:
# Show index information of a collection.
milvus.get_index_info(collection_name=collection_name, timeout=10)

### Get data by ids

In [ ]:
# List the ids in segment
# you can get the segment_name list by get_collection_stats() function.
milvus.list_id_in_segment(collection_name =collection_name, segment_name='1600328539015368000', timeout=None)

In [ ]:
# Returns raw vectors according to ids, and you can get the ids list by list_id_in_segment() function.
# ids (list) -- list of vector id
milvus.get_entity_by_id(collection_name=collection_name, ids=[0], timeout=None)

### Get and set the configurations

In [ ]:
# Gets Milvus configurations.
milvus.get_config(parent_key='cache', child_key='cache_size')

In [ ]:
# Sets Milvus configurations.
milvus.set_config(parent_key='cache', child_key='cache_size', value='5G')

### Delete data/index

In [ ]:
# Removes an index.
milvus.drop_index(collection_name=collection_name, timeout=None)

In [ ]:
# Deletes a partition in a collection.
milvus.drop_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=None)

In [ ]:
# Deletes vectors in a collection by vector ID.
# id_array (list[int]) -- list of vector id
milvus.delete_entity_by_id(collection_name=collection_name, id_array=[0], timeout=None)

In [ ]:
# Deletes a collection by name.
# milvus.drop_collection(collection_name=collection_name, timeout=10)